# String Constants

### Simple string constant

In [ ]:
string name = "John";
string s = String.Empty;

display("-----------------");

s = "Default representation C:\\Test \" ";
display(s);
display("-----------------");

### Ignoring escapes

In [ ]:
s = @"Escapes \ ignored C:\Test "" ";
display(s);
display("-----------------");

### Multiline string

In [ ]:
s = @"
Multiline string
Second line
Third line
";
display(s);
display("-----------------");

### Interpolated string

In [ ]:
s = $"Interpolated |{name}|  |{1+2}|";
display(s);
display("-----------------");

### Interpolated formatted string

In [ ]:
s = $"Interpolated & formated |{name, 5}|  |{1+2,10}|0x{1+2,-10:X3}|";
display(s);
display("-----------------");

### Interpolated formatted multiline string

In [ ]:
s = $@"
Interpolated & formated |{name, 5}|  |{1+2,10}|0x{1+2,-10:X3}|
Second line             |{" ",5}|  |{10+8,10}|0x{10+8,-10:X3}|
";
display(s);
display("-----------------");

# String Concatenation vs format vs interpolation

### Concatenation with "+" operator

In [ ]:
string name = "John";
string lastName = "Johnson";

//Concatenation
var s = "Human " + name + " " + lastName;
display(s);

### Concatenation with String.Format function

In [ ]:
//Format
s = String.Format("Human {0} {1}", name, lastName);
display(s);

### Concatenation using string interpolation

In [ ]:
//Interpolation
s = $"Human {name} {lastName}";
display(s);

# String Builder

In [ ]:
var stringBuilder = new StringBuilder();
stringBuilder.Append("Human ");
stringBuilder.Append(name);
stringBuilder.Append(" ");
stringBuilder.Append(lastName);
var s = stringBuilder.ToString();
display(s);

## Benchmarking

In [ ]:
#r "nuget:BenchmarkDotNet"
using BenchmarkDotNet.Attributes;
using BenchmarkDotNet.Running;
using BenchmarkDotNet.Validators;
using BenchmarkDotNet.Configs;

public class AllowNonOptimized : ManualConfig
{
    public AllowNonOptimized()
    {
        Add(JitOptimizationsValidator.DontFailOnError); // ALLOW NON-OPTIMIZED DLLS

        Add(DefaultConfig.Instance.GetLoggers().ToArray()); // manual config has no loggers by default
        Add(DefaultConfig.Instance.GetExporters().ToArray()); // manual config has no exporters by default
        Add(DefaultConfig.Instance.GetColumnProviders().ToArray()); // manual config has no columns by default
    }
}

public class StringTests
{
    [Params(2, 10, 100, 1_000, 10_000)]
    public int ConcatCount { get; set; }

    [Benchmark]
    public void AsString()
    {
        var s = "";
        for (int i = 0; i < ConcatCount; i++)
        {
            s += "0";
        }
        int len = s.Length;
    }

    [Benchmark]
    public void AsStringBuilder()
    {
        var sb = new StringBuilder();
        for (int i = 0; i < ConcatCount; i++)
        {
            sb.Append("0");
        }
        var s = sb.ToString();
        int len = s.Length;
    }
}

var r = BenchmarkRunner.Run<StringTests>(new AllowNonOptimized());

In [ ]:
cd ..
dotnet run -c Release --project ".\StringBenchmark\StringBenchmark\StringBenchmark.csproj"

|          Method | ConcatCount |                  Mean |                Error |                StdDev |
|---------------- |------------ |----------------------:|---------------------:|----------------------:|
|        AsString |           2 |              29.14 ns |             0.346 ns |              0.324 ns |
| AsStringBuilder |           2 |              51.63 ns |             1.057 ns |              1.646 ns |
|        AsString |          10 |             226.27 ns |             2.469 ns |              2.309 ns |
| AsStringBuilder |          10 |              98.87 ns |             1.564 ns |              1.463 ns |
|        AsString |         100 |           3,972.55 ns |            68.627 ns |             64.194 ns |
| AsStringBuilder |         100 |             903.80 ns |             5.574 ns |              4.941 ns |
|        AsString |        1000 |         186,659.79 ns |         3,698.614 ns |         10,248.842 ns |
| AsStringBuilder |        1000 |           6,855.54 ns |            27.775 ns |             25.981 ns |
|        AsString |       10000 |      16,668,151.62 ns |       338,644.763 ns |        998,501.723 ns |
| AsStringBuilder |       10000 |          65,286.67 ns |           854.459 ns |            799.262 ns |
|        AsString |      100000 |   1,841,048,672.22 ns |    34,370,374.038 ns |     36,775,915.148 ns |
| AsStringBuilder |      100000 |         763,640.93 ns |         6,998.175 ns |          6,546.097 ns |
|        AsString |      200000 |   8,668,458,921.43 ns |    90,338,208.965 ns |     80,082,469.212 ns |
| AsStringBuilder |      200000 |       1,520,557.38 ns |        21,412.503 ns |         20,029.269 ns |
|        AsString |      300000 |  22,151,508,566.67 ns |   127,186,609.624 ns |    118,970,433.555 ns |
| AsStringBuilder |      300000 |       2,406,683.74 ns |        32,016.905 ns |         29,948.633 ns |
|        AsString |      400000 |  44,239,252,650.00 ns |   350,895,219.246 ns |    273,955,906.281 ns |
| AsStringBuilder |      400000 |       3,034,966.69 ns |         9,889.009 ns |          9,250.185 ns |
|        AsString |      500000 |  77,294,167,664.29 ns |   159,666,947.648 ns |    141,540,590.251 ns |
| AsStringBuilder |      500000 |       3,788,364.56 ns |        25,054.498 ns |         23,435.993 ns |
|        AsString |      750000 | 212,266,399,600.00 ns | 4,189,457,499.732 ns |  4,114,607,445.152 ns |
| AsStringBuilder |      750000 |       5,103,197.19 ns |        49,597.841 ns |         46,393.852 ns |
|        AsString |     1000000 | 467,328,417,600.00 ns | 9,237,886,941.572 ns | 15,934,901,088.628 ns |
| AsStringBuilder |     1000000 |       6,895,745.59 ns |        31,462.324 ns |         27,890.531 ns |

| ConcatCount |  Run Time                         |
|-------------|----------------------------------:|
|           2 | Run time: 00:02:02 (122.66 sec)   |
|          10 | Run time: 00:02:25 (145.88 sec)   |
|         100 | Run time: 00:02:45 (165.55 sec)   |
|        1000 | Run time: 00:02:46 (166.2 sec)    |
|       10000 | Run time: 00:02:45 (165.36 sec)   |
|      100000 | Run time: 00:02:40 (160.43 sec)   |
|      200000 | Run time: 00:04:43 (283.07 sec)   |
|      300000 | Run time: 00:09:43 (583.97 sec)   |
|      400000 | Run time: 00:18:21 (1101.85 sec)  |
|      500000 | Run time: 00:29:32 (1772.88 sec)  |
|      750000 | Run time: 01:31:02 (5462.2 sec)   |
|     1000000 | Run time: 05:59:07 (21547.96 sec) |

![StringVsBuilderLog](media/StringVsBuilderLog.png)

![StringVsBuilder](media/StringVsBuilder.png)